In [101]:
import argparse
import boto3
import paramiko
import os, sys, time
from pathlib import Path

In [ ]:
profile = [
    'region': 
]

In [5]:
session = boto3.Session()

In [170]:
ec2 = session.resource('ec2')

In [192]:
c = ec2.vpcs.filter(Filters=[{'Name': 'tag-value', 'Values': ['sdfjklsdf']}])

In [194]:
if list(c):
    print('sdjfklds')

In [186]:
list(ec2.vpcs.filter(Filters=[{'Name': 'tag-value', 'Values': [name]}]))[0].id

TypeError: 'ec2.vpcsCollection' object does not support indexing

In [182]:
list(ec2.vpcs.all())

[ec2.Vpc(id='vpc-6e6b2a17'),
 ec2.Vpc(id='vpc-bc7e3fc5'),
 ec2.Vpc(id='vpc-56b0f02f')]

In [173]:
ec2.vpcs(Filters=[{'Name': 'tag-value', 'Values': [name]}])

TypeError: 'ec2.vpcsCollectionManager' object is not callable

In [7]:
ec2_client = session.client('ec2')

In [39]:
ec2_client.describe_security_groups()

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Wed, 28 Mar 2018 08:38:27 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': 'acf17ac6-1d70-4313-b783-c0146c637c92',
  'RetryAttempts': 0},
 'SecurityGroups': [{'Description': 'default group',
   'GroupId': 'sg-0b31743b',
   'GroupName': 'default',
   'IpPermissions': [{'FromPort': 0,
     'IpProtocol': 'tcp',
     'IpRanges': [],
     'Ipv6Ranges': [],
     'PrefixListIds': [],
     'ToPort': 65535,
     'UserIdGroupPairs': [{'GroupId': 'sg-0b31743b',
       'GroupName': 'default',
       'UserId': '698042698196'}]},
    {'FromPort': 0,
     'IpProtocol': 'udp',
     'IpRanges': [],
     'Ipv6Ranges': [],
     'PrefixListIds': [],
     'ToPort': 65535,
     'UserIdGroupPairs': [{'GroupId': 'sg-0b31743b',
       'GroupName': 'default',
       'UserId': '698042698196'}]},
    {'FromPort': -1,
     'IpProtocol':

In [ ]:
def get_vpc(name):
    vpcs = list(ec2.vpcs.filter(Filters=[{'Name': 'tag-value', 'Values': [name]}]))
    if vpcs:
        return vpcs[0].id
    print('Vpc')
    vpc_req = ec2_client.describe_vpcs(Filters=[{'Name': 'tag-value', 'Values': [name]}])
    try:
        vpc_id = vpc_req['Vpcs'][0]['VpcId']
    except (KeyError, IndexError):
        return None
    return vpc_id

In [37]:
def get_vpc(name):
    vpc_req = ec2_client.describe_vpcs(Filters=[{'Name': 'tag-value', 'Values': [name]}])
    try:
        vpc_id = vpc_req['Vpcs'][0]['VpcId']
    except (KeyError, IndexError):
        return None
    return vpc_id

In [60]:
def get_security_group(name, vpc_id):
    sg_tag = f'{name}-security-group'
    filters = [{'Name': 'group-name', 'Values': [sg_tag]}, {'Name': 'vpc-id', 'Values': [vpc_id]}]
    sg_req = ec2_client.describe_security_groups(Filters=filters)
    try:
        sg_id = sg_req['SecurityGroups'][0]['GroupId']
    except (KeyError, IndexError):
        return None
    return sg_id

In [90]:
def get_subnet(name, vpc_id):
    subnet_tag = f'{name}-subnet'
    filters = [{'Name': 'tag-value', 'Values': [subnet_tag]}, {'Name': 'vpc-id', 'Values': [vpc_id]}]
    subnet_req = ec2_client.describe_subnets(Filters=filters)
    try:
        subnet_id = subnet_req['Subnets'][0]['SubnetId']
    except (KeyError, IndexError):
        return None
    return subnet_id

In [95]:
def get_all_vpc(name):
    vpc_id = get_vpc(name)
    sg_id = get_security_group(name, vpc_id)
    subnet_id = get_subnet(name, vpc_id)
    return vpc_id, sg_id, subnet_id

In [97]:
name='fast-ai'

In [98]:
get_all_vpc(name)

('vpc-6e6b2a17', 'sg-f60fca88', 'subnet-f056ff89')

### Creating a vpc

In [106]:
a = Path.home()

PosixPath('/Users/andrewshaw')

In [140]:
def create_ec2_keypair(name):
    ssh_dir = Path.home()/'.ssh'
    (ssh_dir).mkdir(exist_ok=True)
    keypair_name = f'aws-key-{name}'
    filename = ssh_dir/f'{keypair_name}.pem'
    if filename.exists():
        print('Keypair exists')
        return
    keypair = ec2_client.create_key_pair(KeyName=keypair_name)
    keypair_out = str(keypair['KeyMaterial'])
    outfile = open(filename,'w')
    outfile.write(keypair_out)
    os.chmod(filename, 0o400)
    print('Created keypair')

In [138]:
# create_ec2_keypair('test-fast')

Created keypair


In [ ]:
def create_security_group(name, vpc_id):
    cidr = '0.0.0.0/0'
    sg = ec2_client.create_security_group(GroupName=f'{name}-security-group-test', VpcId=vpc_id, Description='SG for {name} machine')
    sg_id = sg['GroupId']
    # ssh
    ec2_client.authorize_security_group_ingress(GroupId=sg_id, IpProtocol='tcp', FromPort=22, ToPort=22, CidrIp=cidr)
    # jupyter notebook
    ec2_client.authorize_security_group_ingress(GroupId=sg_id, IpProtocol='tcp', FromPort=8888, ToPort=8898, CidrIp=cidr)
    return sg_id

In [157]:
cidr_block='10.0.0.0/28'
vpc = ec2_client.create_vpc(CidrBlock=cidr_block)


In [160]:
vpc

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '865',
   'content-type': 'text/xml;charset=UTF-8',
   'date': 'Wed, 28 Mar 2018 18:23:05 GMT',
   'server': 'AmazonEC2'},
  'HTTPStatusCode': 200,
  'RequestId': '60465ff2-7296-4aa2-9150-86d24569ebb8',
  'RetryAttempts': 0},
 'Vpc': {'CidrBlock': '10.0.0.0/28',
  'CidrBlockAssociationSet': [{'AssociationId': 'vpc-cidr-assoc-90bcb7fb',
    'CidrBlock': '10.0.0.0/28',
    'CidrBlockState': {'State': 'associated'}}],
  'DhcpOptionsId': 'dopt-42213a25',
  'InstanceTenancy': 'default',
  'Ipv6CidrBlockAssociationSet': [],
  'IsDefault': False,
  'State': 'pending',
  'Tags': [],
  'VpcId': 'vpc-56b0f02f'}}

In [164]:
vpc_id = vpc['Vpc']['VpcId']

In [166]:

# Configure the VPC to support DNS resolution and hostname assignment
ec2_client.modify_vpc_attribute(vpc_id, enable_dns_support=True)
ec2_client.modify_vpc_attribute(vpc_id, enable_dns_hostnames=True)

TypeError: modify_vpc_attribute() only accepts keyword arguments.

In [165]:
ec2_client.modify_vpc_attribute(VpcId=vpc_id, EnableDnsSupport=True)

ParamValidationError: Parameter validation failed:
Invalid type for parameter EnableDnsSupport, value: True, type: <class 'bool'>, valid types: <class 'dict'>

In [ ]:

export vpcId=$(aws ec2 create-vpc --cidr-block 10.0.0.0/28 --query 'Vpc.VpcId' --output text)
aws ec2 create-tags --resources $vpcId --tags --tags Key=Name,Value=$name
aws ec2 modify-vpc-attribute --vpc-id $vpcId --enable-dns-support "{\"Value\":true}"
aws ec2 modify-vpc-attribute --vpc-id $vpcId --enable-dns-hostnames "{\"Value\":true}"

In [ ]:
#!/bin/bash

# settings
export name="fast-ai"

export cidr="0.0.0.0/0"
export vpcTag=$name
export sgTag="SG for fast.ai machine"
export subnetTag="$name-subnet"

hash aws 2>/dev/null
if [ $? -ne 0 ]; then
    echo >&2 "'aws' command line tool required, but not installed.  Aborting."
    exit 1
fi

if [ -z "$(aws configure get aws_access_key_id)" ]; then
    echo "AWS credentials not configured.  Aborting"
    exit 1
fi

export vpcId=$(aws ec2 create-vpc --cidr-block 10.0.0.0/28 --query 'Vpc.VpcId' --output text)
aws ec2 create-tags --resources $vpcId --tags --tags Key=Name,Value=$name
aws ec2 modify-vpc-attribute --vpc-id $vpcId --enable-dns-support "{\"Value\":true}"
aws ec2 modify-vpc-attribute --vpc-id $vpcId --enable-dns-hostnames "{\"Value\":true}"

export internetGatewayId=$(aws ec2 create-internet-gateway --query 'InternetGateway.InternetGatewayId' --output text)
aws ec2 create-tags --resources $internetGatewayId --tags --tags Key=Name,Value=$name-gateway
aws ec2 attach-internet-gateway --internet-gateway-id $internetGatewayId --vpc-id $vpcId

export subnetId=$(aws ec2 create-subnet --vpc-id $vpcId --cidr-block 10.0.0.0/28 --query 'Subnet.SubnetId' --output text)
# aws ec2 modify-subnet-attribute --subnet-id $subnetId --map-public-ip-on-launch // Public id needed?
aws ec2 create-tags --resources $subnetId --tags --tags Key=Name,Value=$name-subnet

export routeTableId=$(aws ec2 create-route-table --vpc-id $vpcId --query 'RouteTable.RouteTableId' --output text)
aws ec2 create-tags --resources $routeTableId --tags --tags Key=Name,Value=$name-route-table
export routeTableAssoc=$(aws ec2 associate-route-table --route-table-id $routeTableId --subnet-id $subnetId --output text)
aws ec2 create-route --route-table-id $routeTableId --destination-cidr-block 0.0.0.0/0 --gateway-id $internetGatewayId

export securityGroupId=$(aws ec2 create-security-group --group-name $name-security-group --description $sgTag --vpc-id $vpcId --query 'GroupId' --output text)
# ssh
aws ec2 authorize-security-group-ingress --group-id $securityGroupId --protocol tcp --port 22 --cidr $cidr
# jupyter notebook
aws ec2 authorize-security-group-ingress --group-id $securityGroupId --protocol tcp --port 8888-8898 --cidr $cidr

if [ ! -d ~/.ssh ]
then
	mkdir ~/.ssh
fi

if [ ! -f ~/.ssh/aws-key-$name.pem ]
then
	aws ec2 create-key-pair --key-name aws-key-$name --query 'KeyMaterial' --output text > ~/.ssh/aws-key-$name.pem
	chmod 400 ~/.ssh/aws-key-$name.pem
fi